# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import azureml.core 
print(azureml.core.VERSION)

1.20.0


In [3]:
from azureml.core.environment import Environment
Environment(name="vaenv")
myenv = Environment(name="vaenv")

In [4]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("numpy==1.15.4")
myenv.python.conda_dependencies=dependencies

In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name='Toria_workspace',
            subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029',
            resource_group='udacityprojects',)
exp = Experiment(workspace=ws, name="Toria_workspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Toria_workspace
Azure region: eastus
Subscription id: 784d8740-4c70-4d20-8f45-b2bfecf94029
Resource group: udacityprojects


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_target_name = "vacompute"
compute_cluster = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes = 4)
my_target =  ComputeTarget.create(ws, compute_target_name, compute_cluster)
### YOUR CODE HERE ###

In [7]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
dat = "https://raw.githubusercontent.com/vikkyfama/Project_Capstone/toribranch/Personal_loans.csv"
ds = TabularDatasetFactory.from_delimited_files(dat)
print(ds)

TabularDataset
{
  "source": [
    "https://raw.githubusercontent.com/vikkyfama/Project_Capstone/toribranch/Personal_loans.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [12]:
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn import datasets
from azureml.core.dataset import Dataset
import numpy as np
from traintest2 import clean_data
#from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn import preprocessing


x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y)


Average Experience 20.1046


In [13]:
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn import datasets
from azureml.core.dataset import Dataset
import numpy as np
#from train import clean_data
#from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn import preprocessing
import os


#x_train, x_test, y_train, y_test = train_test_split(P_loan, test_size = 0.3, random_state = 0)
#x_train, x_test, y_train, y_test
training_data = pd.concat([x_train, y_train], axis = 1)

if not os.path.isdir('dataset'):
    os.mkdir('dataset')


training_data.to_csv('dataset/loan.csv')
ds = ws.get_default_datastore()
ds.upload(src_dir='./dataset', target_path='loandataset', overwrite=True, show_progress=True)

training_data = Dataset.Tabular.from_delimited_files(path=ds.path('loandataset/loan.csv'))
training_data

Uploading an estimated of 1 files
Uploading ./dataset/loan.csv
Uploaded ./dataset/loan.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "source": [
    "('workspaceblobstore', 'loandataset/loan.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [14]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'People_loan_experiment'

experiment=Experiment(ws, experiment_name)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
# TODO: Put your automl settings here
automl_settings = {
     "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = 'vacompute',
    task="classification",
    training_data= training_data,
    label_column_name= "Personal Loan",
    n_cross_validations=5
)

In [16]:
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config)
remote_run = experiment.submit(automl_config, show_output = True)
remote_run

Running on remote.
No run_configuration provided, running on vacompute with default configuration
Running on remote compute: vacompute
Parent Run ID: AutoML_5f2a6661-d257-4a23-a2bb-fe58030d4ae4

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one c

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [20]:
best_run, best_model = remote_run.get_output()
best_run, best_model

(Run(Experiment: People_loan_experiment,
 Id: AutoML_5f2a6661-d257-4a23-a2bb-fe58030d4ae4_30,
 Type: azureml.scriptrun,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                                                     min_samples_split=0.2442105263157895,
                                                                                                     m

In [21]:
#TODO: Save the best model
import joblib
joblib.dump(best_model, 'best_automl_model.pkl')

['best_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [49]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('vfenv')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1', 'scikit-learn','sklearn'])

env.python.conda_dependencies = cd
#cd.add_conda_package("scikit-learn")
# Register environment to re-use later
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "vfenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
      

In [22]:
best_run.register_model(model_name = 'best_automl_model.pkl', model_path = './outputs/')


Model(workspace=Workspace.create(name='toria_workspace', subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029', resource_group='udacityprojects'), name=best_automl_model.pkl, id=best_automl_model.pkl:1, version=1, tags={}, properties={})

In [36]:
from azureml.core.model import Model

model = Model.register(model_path="./outputs/",
                       model_name="best_automl_model.pkl",
                       workspace=ws)

Registering model best_automl_model.pkl


TODO: In the cell below, send a request to the web service you deployed to test it.

In [50]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('./outputs/'), 'best_automl_model.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    #return y_hat.tolist()
    return json.dumps(y_hat.tolist())

Overwriting score.py


TODO: In the cell below, print the logs of the web service and delete the service

In [52]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1,
                                               tags={"data": "loans",  "method" : "sklearn"},
                                               description='Predict loans with sklearn'
                                               )

In [53]:

#%%time
#import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'best_automl_model.pkl')


myEnv = Environment.get(workspace=ws, name="vfenv", version="1")
inference_config = InferenceConfig(entry_script="score.py", environment=myEnv)

service_name = 'bestvikrun' #+ str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 53868fdc-4ea1-42d3-a336-6fef4aa30b3f
More information can be found using '.get_logs()'
Error:
{
  "code": "ContainerGroupQuotaReached",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: Resource type 'Microsoft.ContainerInstance/containerGroups' container group quota 'StandardCores' exceeded in region 'eastus'. Limit: '10', Usage: '9' Requested: '1.1'.."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 53868fdc-4ea1-42d3-a336-6fef4aa30b3f
More information can be found using '.get_logs()'
Error:
{
  "code": "ContainerGroupQuotaReached",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: Resource type 'Microsoft.ContainerInstance/containerGroups' container group quota 'StandardCores' exceeded in region 'eastus'. Limit: '10', Usage: '9' Requested: '1.1'.."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Transitioning\nOperation ID: 53868fdc-4ea1-42d3-a336-6fef4aa30b3f\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"ContainerGroupQuotaReached\",\n  \"statusCode\": 400,\n  \"message\": \"ACI Service request failed. Reason: Resource type 'Microsoft.ContainerIns